<a href="https://colab.research.google.com/github/anitayadav3/EmotionRecognitionInConversation/blob/master/ConceptNet_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pickle
import string
import nltk
nltk.download('punkt')
import requests
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
with open('/content/gdrive/My Drive/iemocap/train/sentences.pkl', 'rb') as f:
    data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/train/labels.pkl', 'rb') as f:
    labels = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/sentences.pkl', 'rb') as f:
    test_data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/labels.pkl', 'rb') as f:
    test_labels = pickle.load(f)

In [5]:
def preprocessing(data,labels):
  processed_data=[]
  processed_label=[]
  for i in range(0,len(data)):
    for j in range(0,len(data[i])):
      intermediate_data=[]
      intermediate_label=[]
      for k in range(0,len(data[i][j])):
        text=data[i][j][k]
        if text != '<eos>'and text!='<pad>':
          intermediate_data.append(text)
      processed_data.append(intermediate_data)
  for i in labels:
    for j in i:
      processed_label.append(j)
  return processed_data,processed_label

In [41]:
processed_data,processed_label = preprocessing(data,labels)
test_processed_data,test_processed_label = preprocessing(test_data,test_labels)

In [42]:
for i in range(0,len(processed_data)):
  processed_data[i]= ' '.join(processed_data[i])
for i in range(0,len(test_processed_data)):
  test_processed_data[i]=' '.join(test_processed_data[i])

In [38]:
conceptnet_data=[]
temp=processed_data[0]
vars=temp.translate(str.maketrans('', '', string.punctuation))
tokens=nltk.word_tokenize(vars)
for i in tokens:
  obj = requests.get('http://api.conceptnet.io/related/c/en/' + i + '?filter=/c/en').json()
  for j in range(1,3):
    var=obj['related'][j]['@id']
    ans=re.sub(r'[^\w]', ' ', var)
    ans=ans[6:]
    conceptnet_data.append(ans)
for i in conceptnet_data:
  processed_data[0]=processed_data[0]+" "+str(i)

In [49]:
new_process=[]
for i in range(0,2):
  new_process.append(processed_data[i])

In [47]:
def add_ConceptNet_Features(data):
  print(len(data))
  for sentences in range(0,len(data)):
    conceptnet_data=[]
    cleaned_sentences=data[sentences].translate(str.maketrans('', '', string.punctuation))
    tokens=nltk.word_tokenize(cleaned_sentences)
    for i in tokens:
      obj = requests.get('http://api.conceptnet.io/related/c/en/' + i + '?filter=/c/en').json()
      for j in range(1,3):
        response=obj['related'][j]['@id']
        response=re.sub(r'[^\w]', ' ', response)
        response=response[6:]
        conceptnet_data.append(response)
    for i in conceptnet_data:
      print(data[sentences])
      print(str(i))
      data[sentences]=data[sentences]+" "+str(i)
  return data

In [52]:
new_data = add_ConceptNet_Features(new_process)

2
thank you for calling sprint . we care about everybody . how can i help you ?
thanked
thank you for calling sprint . we care about everybody . how can i help you ? thanked
thank_you
thank you for calling sprint . we care about everybody . how can i help you ? thanked thank_you
y_all
thank you for calling sprint . we care about everybody . how can i help you ? thanked thank_you y_all
you ve
thank you for calling sprint . we care about everybody . how can i help you ? thanked thank_you y_all you ve
forthe
thank you for calling sprint . we care about everybody . how can i help you ? thanked thank_you y_all you ve forthe
herefor
thank you for calling sprint . we care about everybody . how can i help you ? thanked thank_you y_all you ve forthe herefor
calls
thank you for calling sprint . we care about everybody . how can i help you ? thanked thank_you y_all you ve forthe herefor calls
call
thank you for calling sprint . we care about everybody . how can i help you ? thanked thank_you y_al